In [2]:
import torch

# Confirm that the GPU is detected
assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


In [3]:
!pip install transformers
!pip install -U -q PyDrive
!pip install opendatasets
!pip install pandas
!pip install accelerate -U
!pip install datasets evaluate
!pip install warcio selectolax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 17.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [4]:
from datasets import load_dataset

dataset = load_dataset("snoop2head/commoncrawl_sampled_gpt2-xl")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/81949 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/snoop2head___parquet/snoop2head--commoncrawl_sampled_gpt2-xl-2086808043a3fe75/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset = dataset["train"].select(range(int(len(dataset["train"])//4))).train_test_split(test_size=0.2)

In [6]:
from transformers import DataCollatorForLanguageModeling
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import AutoTokenizer, GPTNeoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")
model.cuda()


GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

In [7]:
max_length = tokenizer.model_max_length
print(max_length)
def preprocess_function(examples):
  #print(len(examples["generated"]))
  #print(len([" ".join(x) for x in examples["generated"]]))
  text = [" ".join(x) for x in examples["generated"]]
  return tokenizer(text,truncation=True, max_length=2048)

2048


In [8]:
datamap = dataset.map(
    preprocess_function,
    batched=True,
    num_proc = 8,
    remove_columns=dataset["train"].column_names,
)


Map (num_proc=8):   0%|          | 0/16389 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/4098 [00:00<?, ? examples/s]

In [9]:
print(datamap["train"][0])

{'input_ids': [43, 1312, 264, 256, 1312, 299, 308, 220, 220, 267, 277, 220, 220, 327, 267, 334, 299, 256, 331, 220, 220, 360, 304, 279, 257, 374, 256, 285, 304, 299, 256, 1220, 440, 277, 277, 1312, 269, 304, 220, 220, 350, 289, 267, 299, 304, 220, 220, 399, 334, 285, 275, 304, 374, 264, 220, 220, 277, 267, 374, 220, 220, 256, 289, 304, 220, 220, 277, 267, 300, 300, 267, 266, 1312, 299, 308, 220, 220, 269, 267, 334, 299, 256, 1312, 304, 264, 1058, 220, 198, 220, 198, 317, 300, 257, 285, 304, 288, 257, 837, 220, 220, 317, 300, 269, 267, 299, 257, 837, 220, 220, 317, 285, 257, 288, 267, 374, 837, 220, 220, 347, 334, 256, 256, 304, 837, 220, 220, 327, 257, 300, 257, 410, 304, 374, 257, 264, 837, 220, 220, 327, 267, 300, 334, 264, 257, 837, 220, 220, 327, 267, 299, 256, 374, 257, 220, 220, 327, 267, 264, 256, 257, 837, 220, 220, 360, 304, 300, 220, 220, 399, 267, 374, 256, 304, 837, 220, 220, 412, 300, 220, 220, 360, 267, 374, 257, 288, 267, 837, 220, 220, 376, 374, 304, 264, 299, 267, 837,

In [10]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
lm_dataset = datamap.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/16389 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4098 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
training_args = TrainingArguments(
        output_dir="crawl-model",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_gpu_train_batch_size=24,
        weight_decay=0.01,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.324400,1.319927


In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

In [ ]:
trainer.save_model("model1")

In [ ]:
!zip -r /content/model1.zip /content/model1

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from google.colab import files
files.download('model1.zip')